In [3]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

True

In [4]:
model = init_chat_model(
    "google_genai:gemini-3-flash-preview",
    api_key=os.getenv("GOOGLE_API_KEY"),
    max_output_tokens=1000
)

Integrating tools

In [5]:
#web_search
import os
from langchain.tools import tool
from tavily import TavilyClient

api_key = os.getenv("TAVILY_API_KEY")

@tool("web_search")
def search(query: str) -> str:
    """search the web for information."""
    tavily_client = TavilyClient(api_key=api_key)
    response = tavily_client.search(query=query, max_results=1)
    return response["results"][0]["content"]

result = search.invoke({"query": "latest advancements in artificial intelligence"})
print(result)

AI and ML advancements are transforming engineering by automating complex tasks and enhancing decision-making processes for professionals.


In [6]:
#calculator
@tool(
    "calculator",
    description="Performs arithmetic calculations. Use this for any math problems."
)
def calc(expression: str) -> str:
    """
    Evaluate mathematical expressions..
    """
    try:
        result = eval(expression, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


In [7]:
calc.invoke({"expression": "20 * 4 / 5"})

'16.0'

In [8]:
#weather
from pydantic import BaseModel, Field
from typing import Literal
from langchain.tools import tool
from tavily import TavilyClient

tavily_client = TavilyClient()

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include weather forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(
    location: str,
    units: str = "celsius",
    include_forecast: bool = False
) -> str:
    """Get current weather and optional forecast."""
    
    query = f"current weather in {location}"
    if include_forecast:
        query += " 5 day forecast"

    response = tavily_client.search(query=query, max_results=1)
    content = response["results"][0]["content"]

    return f"Weather information for {location} ({units}):\n{content}"


In [9]:
get_weather.invoke({
    "location": "Hyderabad",
    "units": "celsius",
    "include_forecast": True
})

"Weather information for Hyderabad (celsius):\n{'location': {'name': 'Hyderabad', 'region': 'Telangana', 'country': 'India', 'lat': 17.3753, 'lon': 78.4744, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1768125395, 'localtime': '2026-01-11 15:26'}, 'current': {'last_updated_epoch': 1768124700, 'last_updated': '2026-01-11 15:15', 'temp_c': 25.1, 'temp_f': 77.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.3, 'wind_kph': 10.1, 'wind_degree': 139, 'wind_dir': 'SE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 41, 'cloud': 50, 'feelslike_c': 24.6, 'feelslike_f': 76.3, 'windchill_c': 23.4, 'windchill_f': 74.1, 'heatindex_c': 23.7, 'heatindex_f': 74.6, 'dewpoint_c': 3.9, 'dewpoint_f': 39.0, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 1.6, 'gust_mph': 7.5, 'gust_kph': 12.1}}"

Helper Functions

In [10]:
#format_result
import os
from langchain.tools import tool
from tavily import TavilyClient

api_key = os.getenv("TAVILY_API_KEY")

@tool("web_search")
def search(query: str) -> str:
    """search the web for information."""
    tavily_client = TavilyClient(api_key=api_key)
    response = tavily_client.search(query=query, max_results=1)
    return response

def format_result(result):
    """
    Extract and return only content, title, and url from Tavily response.
    """
    item = result["results"][0]
    return {
        "content": item["content"],
        "title": item["title"],
        "url": item["url"]
    }

tavily_response = search.invoke({"query": "latest advancements in artificial intelligence"})
formatted_result = format_result(tavily_response)
print(formatted_result)

{'content': 'AI and ML advancements are transforming engineering by automating complex tasks and enhancing decision-making processes for professionals.', 'title': 'Advancements in AI and Machine Learning', 'url': 'https://ep.jhu.edu/news/advancements-in-ai-and-machine-learning/'}


In [15]:
#summarise_web
def summarise_web(user_query: str) -> str:
    prompt1 = f"""
Convert this into a short and precise web search query:
{user_query}
"""
    response1 = model.invoke(prompt1)

    if isinstance(response1.content, list):
        refined_query = response1.content[0]["text"].strip()
    else:
        refined_query = response1.content.strip()

    print("Refined query:", refined_query)

    web_result = search.invoke({"query": refined_query})
    print("\nWeb search result:\n", web_result)

    prompt2 = f"""
Summarise the following information in 3 short lines.
Focus only on the most important points and avoid extra details.

Content:
{web_result}
"""
    response2 = model.invoke(prompt2)

    if isinstance(response2.content, list):
        final_summary = response2.content[0]["text"].strip()
    else:
        final_summary = response2.content.strip()

    return final_summary

summary = summarise_web(
    "Explain about corona virus"
)
print("\nFinal Summary:\n", summary)


Refined query: **COVID-19 overview**

Web search result:
 {'query': '**COVID-19 overview**', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC7205601/', 'title': 'An overview of COVID-19 - PMC - PubMed Central', 'content': 'COVID-19 is the result of infection with severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) that was first isolated and identified in patients who were exposed at a seafood market in Wuhan City, Hubei Province, China on December 2019 (Zhu N et al., 2020). Patients with underlying co-morbidities (including hypertension, diabetes, pre-existing respiratory infection, cardiovascular disease, and cancer) are more likely to succumb and undergo progression to the more severe forms of COVID-19 and also have a higher risk of developing complications (Guan et al., 2020; Special Expert Group for Control of the Epidemic of Novel Coronavirus Pneumonia of the Chinese Preventive Medicine Association,

In [16]:
#get_today_date
from datetime import datetime

def get_today_date():
    """
    Returns today's date in d:m:y format
    """
    today = datetime.now()
    return f"{today.day}:{today.month}:{today.year}"

print(get_today_date())


11:1:2026


In [18]:
#deduplication
def deduplication(results):
    """
    Remove duplicate search results based on URL.
    """
    unique_results = []
    seen = set()

    for item in results:
        url = item.get("url")
        if url and url not in seen:
            seen.add(url)
            unique_results.append(item)

    return unique_results

@tool
def search(query: str):
    """Web search and return unique results."""
    tavily_client = TavilyClient(api_key=api_key)
    response = tavily_client.search(query=query, max_results=5)
    results = response.get("results", [])

    if not results:
        return "No results found."

    unique_results = deduplication(results)
    return unique_results
print(search.invoke({"query": "recent advancements in generative ai"}))

[{'url': 'https://www.aidataanalytics.network/data-science-ai/articles/advancements-in-generative-ai', 'title': 'Advancements in generative AI', 'content': 'In 2024, an AI-generated painting sold at auction for more than US$1 million. Meanwhile, a novel drug molecule discovered by a generative AI', 'score': 0.82728493, 'raw_content': None}, {'url': 'https://www.nice.com/info/understanding-generative-ai-the-future-of-creative-artificial-intelligence', 'title': 'Understanding generative ai: the future of creative ... - NiCE', 'content': 'Generative AI has experienced significant growth with new tools and technological advances following the introduction of ChatGPT in November 2022. Notable', 'score': 0.78611463, 'raw_content': None}, {'url': 'https://arxiv.org/abs/2407.14962', 'title': 'Recent Advances in Generative AI and Large Language ...', 'content': 'by DH Hagos · 2024 · Cited by 156 — This paper explores the current state of these cutting-edge technologies, demonstrating their rema